In [ ]:
import pickle
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import copy
import numpy as np

from helpers import (
    PSeq_Base,
    PSeq_Params,
    PSeq_Moments3D,
    PSeq_Excite_PE,
    PSeq_TestWave_Chirp,
)

In [12]:
# Setup base sequernce parameters
params = PSeq_Params(channels=('x', 'y', 'z'), max_grad=40, max_slew=140)
pseq0 = PSeq_Base(params)

pseq_excite = PSeq_Excite_PE(params)

# Setup chirp test waveforms
pseq_test = PSeq_TestWave_Chirp(params, do_adc=True)

pseq_refocus = PSeq_Moments3D(
    params,
    start_areas=(None, None, pseq_test.all_areas),
    end_areas=(0, 0, 0),
    start_polarities=(1, 1, (-1, 0, 1)),
)

In [ ]:
N_av = 4
TR = 500e-3

# Slice offsets
FOVz = 80e-3  # 8 cm
Nslices = 5
slice_shift = 1e-3
all_offsets = np.linspace(-FOVz/2, FOVz/2, Nslices) + slice_shift

all_meas = []

pseq0.reinit_seq()
for i_av in range(N_av):
    for channels in [("y", "z", "x"), ("x", "z", "y"), ("x", "y", "z")]:
        pseq0.channels = channels
            
        for wave_idx in range(pseq_test.N_waves):
            for polarity in [-1, 0, 1]:
                for offset in all_offsets:
                
                    pseq0.channels = channels

                    dur = 0
                    dur += pseq0.add_pseq_to_self(pseq_excite, offset=offset)
                    dur += pseq0.add_pseq_to_self(pseq_test, idx=wave_idx, polarity=polarity)
                    dur += pseq0.add_pseq_to_self(pseq_refocus, idx2=wave_idx, start_pol2=polarity)

                    pseq0.add_delay(TR-dur)
                    pseq0.increment_rf_spoiling()
                    
                    all_meas.append(
                        {'channels': channels,
                        'wave_idx': wave_idx,
                        'polarity': polarity,
                        'i_av': i_av,
                        },
                    )

[-39. -19.   1.  21.  41.]


In [14]:
seq_name = f'phantom_girf_v1_{N_av}av'
print(f'======== {seq_name} ========')

pseq0.seq.set_definition(key="Name", value=seq_name)
pseq0.seq.set_definition(key="FOV", value=[100e-3, 100e-3, FOVz])

pseq0.check()

save_res = {'all_meas': all_meas}
Path('./seqs/').mkdir(parents=True, exist_ok=True)
pseq0.seq.write(f'./seqs/{seq_name}.seq')
with Path(f'./seqs/{seq_name}.pickle').open('wb') as handle:
    pickle.dump(save_res, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(' * SAVED! *')

======== phantom_girf_v1_4av ========
Seq Time: 0:18:00
Timing check passed successfully
 * SAVED! *
